## 1. 라이브러리 및 데이터 준비

In [1]:
import torch
import torch.nn as nn
import numpy as np

- 아래는 nn.lstm의 output, hidden state에 대해 설명함.  
  (https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)  
- seq2seq 구현 참고 자료:  
  (https://codlingual.tistory.com/91)

In [28]:
# 학습 데이터
in_text = "nice to meet you"
out_text = "ravi de vous rencontrer"
#de_input = "<sos> ravi de vous rencontrer"
#out_target = "ravi de vous rencontrer <eos>"
# 문자 집합 생성
chars = set(text.split())
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}


In [35]:
outs = set(out_text.split()) 
out_to_index = {char: i+1 for i, char in enumerate(outs)}
out_to_index['<sos>'] = 0
out_to_index['<eos>'] = 5

index_to_out = {i: key for key, i in out_to_index.items()}


In [33]:
out_to_index

{'ravi': 1, 'rencontrer': 2, 'vous': 3, 'de': 4, '<sos>': 0, '<eos>': 5}

In [36]:
index_to_out

{1: 'ravi', 2: 'rencontrer', 3: 'vous', 4: 'de', 0: '<sos>', 5: '<eos>'}

In [19]:
# 하이퍼파라미터
sequence_length = 4  # 입력 길이
vocab_size = len(chars)  # 문자 집합 크기
embedding_dim = 10  # 임베딩 차원
hidden_size = 128  # LSTM 은닉 상태 크기
num_layers = 1  # LSTM 레이어 개수

In [10]:
for key, value in index_to_char.items():
    print(key,value)

0 nice
1 to
2 you
3 meet


## 2. 데이터 전처리 (One-Hot Encoding)

In [13]:
# One-Hot Encoding 함수
def one_hot_encode(indices, vocab_size):
    one_hot = np.zeros((len(indices), vocab_size), dtype=np.float32)
    for i, idx in enumerate(indices):
        one_hot[i, idx] = 1.0
    return one_hot

In [56]:
# 입력과 출력 데이터 생성
input_sequences = []

for i, _ in index_to_char.items():
    input_sequences.append(i)

In [58]:
input_sequences

[0, 1, 2, 3]

In [45]:
out1 = ('<sos> ' + out_text).split()
out2 = (out_text + ' <eos>').split()

In [46]:
out2

['ravi', 'de', 'vous', 'rencontrer', '<eos>']

In [50]:
#input_decode = []
input_decode = [out_to_index[x] for x in out1]
target = [out_to_index[x] for x in out2]
    

In [52]:
target

[1, 4, 3, 2, 5]

In [60]:
# One-Hot Encoding 적용
x = one_hot_encode(input_sequences,4)


In [62]:
x.shape

(4, 4)

In [ ]:
y = np.array(output_sequences)

# Tensor 변환
x_encoder = torch.tensor(x, dtype=torch.float32)  # 입력은 Float Tensor
x_decoder = torch.tensor(y, dtype=torch.long)  # 출력은 Long Tensor